In [ ]:
!pip install lifelines
!pip install pyckmeans
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=84d2ec206762eed94534c5d8ccd6b0c9ab07ecf9fcd7e36287f07bffebcf6e88
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyckmeans: filename=pyckmeans-0.9.4-cp310-cp310-linux_x86_64.whl size=93217 sha256=883a492698b0c9b43b52181b43f085535be0aa305e8a3f0ff2ca32ec6fc4c4e5
  Stored in directory: /root/.cache/pip/wheels/c3/83/bb/d0fc31d29f361aff1f400ca2fb2d32c98f900e1aa480229f99
Successfully built pyckmeans
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s

In [ ]:
import numpy as np
import pandas as pd
from lifelines.fitters.coxph_fitter import CoxPHFitter
from pyckmeans import CKmeans
import matplotlib.pyplot as plt

from google.colab import drive

import os
import wandb

In [ ]:
# --- Load raw BRCA data ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [ ]:
data_frame = pd.read_csv("TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv", sep="\t")
print("Original dataframe shape:", data_frame.shape)
data_frame = data_frame[['age', 'OS', 'OS.time', 'stage_2', 'stage_3', 'stage_4']]

cluster_column = np.loadtxt("VAE_Z_EM_Mean_4_0.txt")
print("Cluster column shape:", cluster_column.shape)
cluster_frame = pd.DataFrame({'cluster': cluster_column})


# Concatenate cluster membership to data table
if 'cluster' in data_frame.columns.values:
  data_frame = data_frame.drop('cluster', axis = 1) # Drop columns that could have been created previously
data_frame = pd.concat([data_frame, cluster_frame], axis = 1)

# Get rows with null entry
null_mask = data_frame.isnull().any(axis=1)
null_rows = data_frame[null_mask]

# Remove rows with null entry
print(data_frame.shape)
data_frame.dropna(inplace=True) # remove all rows with any null value
print(data_frame.shape)

# Fit Cox model
cph = CoxPHFitter()
cph.fit(data_frame, duration_col = 'OS.time', event_col = 'OS')
cph.print_summary()

p_values = cph._compute_p_values()

print("p-value cluster: ", p_values[4])

Original dataframe shape: (1079, 18)
Cluster column shape: (1079,)
(1079, 7)
(1054, 7)


<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -745.33
         time fit was run = 2024-03-15 10:40:47 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.59       1.81       0.28             0.04             1.15                 1.04                 3.14
stage_3     1.35       3.86       0.30             0.77             1.93                 2.16                 6.90
stage_4     2.46      11.70       0.37             1.74             3.18                 5.69                24.04
cluster     1.30       3.68       0.40             0.53             2.08                 1.69                 7.98

            cmp to    z      p   -log2(p)
covariate                                
age           0.00 5.40 <0.005      23.87
stage_2       0.00 2.12   0.03       4.86
stage_3       0.00 4.57 <0.005      17.62
stage_4       0.00 6.69 <0.005      35.39
cluster       0.00 3.29 <0.005       9.98
---
Concordance = 0.76
Partial AIC = 1500.66
log-likelihood ratio test = 83.26 on 5 df
-log2(p) of ll-ratio test = 52.35

p-value cluster:  0.0009905756738502187
Cluster index:  1


In [ ]:
num_lumA = 0
num_lumB = 0
num_basal = 0
num_her2 = 0
num_normal = 0

list_of_indices = []
for j in range(0, len(list(cluster_column))):
  if cluster_column[j] == 1:
    list_of_indices.append(j)
print("List of indices:", list_of_indices)
print("Size of subgroup:", len(list_of_indices))

subtype_data_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv", sep="\t")
#subtype_data_frame.head()
subtype_labels = subtype_data_frame.values[:,1]
print(subtype_labels)

for k in range(0, len(list_of_indices)):
  if subtype_labels[list_of_indices[k]] == 'LumA':
    num_lumA = num_lumA + 1
  elif subtype_labels[list_of_indices[k]] == 'LumB':
    num_lumB = num_lumB + 1
  elif subtype_labels[list_of_indices[k]] == 'Basal':
    num_basal = num_basal + 1
  elif subtype_labels[list_of_indices[k]] == 'Her2':
    num_her2 = num_her2 + 1
  elif subtype_labels[list_of_indices[k]] == 'Normal':
    num_normal = num_normal + 1

print("---------")
print("LumA: ", num_lumA)
print("LumB: ", num_lumB)
print("Basal: ", num_basal)
print("Her2: ", num_her2)
print("Normal: ", num_normal)
sum = num_lumA + num_lumB + num_basal + num_her2 + num_normal
print("Summe: ", sum)

List of indices: [8, 22, 113, 115, 133, 138, 139, 141, 162, 206, 212, 262, 300, 313, 374, 385, 510, 599, 763, 767, 771, 857, 877, 879, 913, 979, 982, 993, 1014, 1018, 1019, 1020, 1024, 1030, 1069, 1070]
Size of subgroup: 36
['LumB' 'Her2' 'LumB' ... 'LumA' 'LumA' 'LumB']
---------
LumA:  0
LumB:  2
Basal:  24
Her2:  7
Normal:  3
Summe:  36
